# 3D CNN 模型用于 MRI 影像处理

本 Notebook 展示了如何使用 3D CNN 模型来处理 MRI 影像，以判定是否患有阿尔茨海默病（AD）。

## 数据加载和预处理


In [5]:
import pandas as pd
import numpy as np
import nibabel as nib  # 用于读取 NIFTI 文件
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# 加载 CSV 文件
data_df = pd.read_csv('structured_data.csv')

# 初始化 X (影像) 和 y (标签) 的列表
X = []
y = []

# 加载和预处理影像
for _, row in data_df.iterrows():
    image_path = row['Image Path']
    image = nib.load(image_path).get_fdata()
    # TODO: 添加必要的影像预处理步骤
    X.append(image)
    y.append(row['Label'])

X = np.array(X)
y = np.array(y)

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


ModuleNotFoundError: No module named 'tensorflow'

## 定义 3D CNN 模型
以下是一个基础的 3D CNN 模型结构。


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense

def build_model():
    model = Sequential([
        Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(x, y, z, 1)),  # 替换 x, y, z 为影像的实际维度
        MaxPooling3D(pool_size=(2, 2, 2)),
        Conv3D(64, kernel_size=(3, 3, 3), activation='relu'),
        MaxPooling3D(pool_size=(2, 2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_model()


## 训练模型
我们将在训练集上训练模型，并使用一部分数据作为验证集。


In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=10, validation_split=0.2)


## 评估模型
最后，我们将在测试集上评估模型的性能。


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")
